In [9]:
pip install extract-msg

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
import extract_msg

input_folder = r"C:\Users\shril\Downloads\Mail loads"
output_folder = r"/MAJOR PROJECT/mail op msg"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".msg"):
        msg = extract_msg.Message(os.path.join(input_folder, filename))
        with open(os.path.join(output_folder, filename.replace('.msg', '.eml')), 'w', encoding='utf-8') as f:
            f.write(msg.as_string())


In [6]:
import os
print(os.getcwd())

C:\Users\shril\MAJOR PROJECT


In [14]:
import email
import os

# Specify the directory where your .eml files are saved
email_directory = "C:\\Users\\shril\\Downloads\\Mail loads"  # Change to your folder

# Function to process .eml files
def process_eml(file_path):
    with open(file_path, "rb") as f:
        msg = email.message_from_bytes(f.read())
        
        # Extract subject and sender
        subject, encoding = email.header.decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding or "utf-8")
        
        from_ = msg.get("From")
        
        # Extract body (plain text or HTML)
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))
                
                if "attachment" not in content_disposition:
                    if content_type == "text/plain":
                        body = part.get_payload(decode=True).decode()  # Decode text
                        break
                    elif content_type == "text/html":
                        body = part.get_payload(decode=True).decode()  # Decode HTML
                        break
        else:
            body = msg.get_payload(decode=True).decode()  # For non-multipart emails

        # Print extracted information
        print(f"From: {from_}")
        print(f"Subject: {subject}")
        print(f"Body: {body[:200]}...")  # Print first 200 chars of body
        print("-" * 50)

# Process all .eml files in the directory
for file_name in os.listdir(email_directory):
    if file_name.endswith(".eml"):
        process_eml(os.path.join(email_directory, file_name))


From: Dean - Academics and IQAC <iqac@cmrit.ac.in>
Subject: Fwd: Applications Open: DMP 2025 Open-Source Mentorship Program | PPO
 for Top Performers
Body: ---------- Forwarded message ---------
From: Poonam Tijare <director.placement@cmrit.ac.in>
Date: Wed, Apr 23, 2025 at 8:08 PM
Subject: Fwd: Applications Open: DMP 2025 Open-Source Mentorship Progr...
--------------------------------------------------
From: Instructables <no-reply@mail.newsletter.instructables.com>
Subject: Autonomous Home Robot
Body: <html xmlns="http://www.w3.org/1999/xhtml"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta conten...
--------------------------------------------------
From: Vice Principal CMRIT <viceprincipal@cmrit.ac.in>
Subject: Fwd: Cultura’25 is Here! Join the Ultimate Techno-Cultural Fest on April 25 & 26!
Body: Dr B Narasimha Murthy
Vice Principal, CMRIT
Bangalore

---------- Forwarded message

In [16]:
import pandas as pd

# Initialize an empty list to store email data
email_data = []

# Modify the process function to store data
def process_eml(file_path):
    with open(file_path, "rb") as f:
        msg = email.message_from_bytes(f.read())
        
        # Extract information
        subject, encoding = email.header.decode_header(msg["Subject"])[0]
        if isinstance(subject, bytes):
            subject = subject.decode(encoding or "utf-8")
        
        from_ = msg.get("From")
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))
                
                if "attachment" not in content_disposition:
                    if content_type == "text/plain":
                        body = part.get_payload(decode=True).decode()  # Decode text
                        break
                    elif content_type == "text/html":
                        body = part.get_payload(decode=True).decode()  # Decode HTML
                        break
        else:
            body = msg.get_payload(decode=True).decode()  # For non-multipart emails

        # Append data to the list
        email_data.append({"From": from_, "Subject": subject, "Body": body[:200]})

# Process all .eml files in the directory
for file_name in os.listdir(email_directory):
    if file_name.endswith(".eml"):
        process_eml(os.path.join(email_directory, file_name))

# Create a DataFrame from the list
df = pd.DataFrame(email_data)

# Save to CSV (optional)
df.to_csv("emails.csv", index=False)
print("Data saved to emails.csv")


Data saved to emails.csv


In [17]:
pip install pandas beautifulsoup4 lxml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import email
from email import policy
from bs4 import BeautifulSoup
import pandas as pd
import re

# Path to folder containing .eml files
eml_folder = r"C:\Users\shril\Downloads\Mail loads"
email_data = []

def extract_links(body):
    return re.findall(r'https?://\S+', body)

# Loop through each .eml file
for filename in os.listdir(eml_folder):
    if filename.endswith(".eml"):
        with open(os.path.join(eml_folder, filename), 'r', encoding='utf-8') as f:
            msg = email.message_from_file(f, policy=policy.default)

            from_email = msg.get("From")
            to_email = msg.get("To")
            subject = msg.get("Subject", "")
            date = msg.get("Date")

            body = ""
            is_html = 0
            num_attachments = 0

            for part in msg.walk():
                content_type = part.get_content_type()
                content_disposition = str(part.get("Content-Disposition"))

                if content_type == "text/plain" and "attachment" not in content_disposition:
                    body = part.get_payload(decode=True).decode(errors='ignore')
                elif content_type == "text/html":
                    is_html = 1
                    html = part.get_payload(decode=True).decode(errors='ignore')
                    body = BeautifulSoup(html, "lxml").get_text()
                elif "attachment" in content_disposition:
                    num_attachments += 1

            links = extract_links(body)
            email_data.append({
                "from_email": from_email,
                "to_email": to_email,
                "subject": subject,
                "subject_keywords": " ".join(subject.lower().split()[:5]),  # top 5 words
                "timestamp": date,
                "body_text": body[:300],  # optionally truncate
                "num_links": len(links),
                "num_attachments": num_attachments,
                "is_html": is_html
            })


In [3]:
df = pd.DataFrame(email_data)
df.to_csv("emails_features.csv", index=False)

In [6]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

# Use the correct variable name: email_data
for email_entry in email_data:
    from_node = email_entry["from_email"]
    to_node = email_entry["to_email"]
    subject = email_entry["subject"]

    if from_node and to_node:
        # Add nodes and edge
        G.add_node(from_node)
        G.add_node(to_node)
        G.add_edge(from_node, to_node, subject=subject)
# Show graph info
print("✅ Graph created")
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Show a few edges with subjects
print("\nSample edges:")
for u, v, d in list(G.edges(data=True))[:5]:
    print(f"{u} -> {v}, Subject: {d['subject']}")

✅ Graph created
Number of nodes: 17
Number of edges: 12

Sample edges:
Dean - Academics and IQAC <iqac@cmrit.ac.in> -> All Students <allstudents@cmrit.ac.in>, All Staff <allstaff@cmrit.ac.in>, Subject: Fwd: Subject: Call for Abstracts – Poster Presentation Awards at 3D GEM 2025 (5th Edition) | IISc Bengaluru
Dean - Academics and IQAC <iqac@cmrit.ac.in> -> All Staff <allstaff@cmrit.ac.in>, All Students <allstudents@cmrit.ac.in>, Subject: Fwd: 🏆 CSE Students Win GCEM Hacks 2K25 Hackathon – Congratulations!
Instructables <no-reply@mail.newsletter.instructables.com> -> shnk22aiml@cmrit.ac.in, Subject: Autonomous Home Robot
Vice Principal CMRIT <viceprincipal@cmrit.ac.in> -> All Staff <allstaff@cmrit.ac.in>, All Students <allstudents@cmrit.ac.in>, Subject: Fwd: Cultura’25 is Here! Join the Ultimate Techno-Cultural Fest on April 25 & 26!
Vice Principal CMRIT <viceprincipal@cmrit.ac.in> -> All Students <allstudents@cmrit.ac.in>, Subject: Fwd: circular


In [10]:
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Print a few edges
for u, v, d in list(G.edges(data=True))[:5]:
    print(f"{u} -> {v}, Subject: {d['subject']}")

Number of nodes: 17
Number of edges: 12
Dean - Academics and IQAC <iqac@cmrit.ac.in> -> All Students <allstudents@cmrit.ac.in>, All Staff <allstaff@cmrit.ac.in>, Subject: Fwd: Subject: Call for Abstracts – Poster Presentation Awards at 3D GEM 2025 (5th Edition) | IISc Bengaluru
Dean - Academics and IQAC <iqac@cmrit.ac.in> -> All Staff <allstaff@cmrit.ac.in>, All Students <allstudents@cmrit.ac.in>, Subject: Fwd: 🏆 CSE Students Win GCEM Hacks 2K25 Hackathon – Congratulations!
Instructables <no-reply@mail.newsletter.instructables.com> -> shnk22aiml@cmrit.ac.in, Subject: Autonomous Home Robot
Vice Principal CMRIT <viceprincipal@cmrit.ac.in> -> All Staff <allstaff@cmrit.ac.in>, All Students <allstudents@cmrit.ac.in>, Subject: Fwd: Cultura’25 is Here! Join the Ultimate Techno-Cultural Fest on April 25 & 26!
Vice Principal CMRIT <viceprincipal@cmrit.ac.in> -> All Students <allstudents@cmrit.ac.in>, Subject: Fwd: circular


In [13]:
import pandas as pd

# Define nodes
nodes = [
    "iqac@cmrit.ac.in", "allstudents@cmrit.ac.in", "allstaff@cmrit.ac.in",
    "no-reply@mail.newsletter.instructables.com", "shnk22aiml@cmrit.ac.in",
    "viceprincipal@cmrit.ac.in"
]

# Make sure all nodes are unique
nodes_df = pd.DataFrame({"id": list(set(nodes))})

# Define edges (sender -> receiver)
edges = [
    ("iqac@cmrit.ac.in", "allstudents@cmrit.ac.in"),
    ("iqac@cmrit.ac.in", "allstaff@cmrit.ac.in"),
    ("iqac@cmrit.ac.in", "allstaff@cmrit.ac.in"),
    ("iqac@cmrit.ac.in", "allstudents@cmrit.ac.in"),
    ("no-reply@mail.newsletter.instructables.com", "shnk22aiml@cmrit.ac.in"),
    ("viceprincipal@cmrit.ac.in", "allstaff@cmrit.ac.in"),
    ("viceprincipal@cmrit.ac.in", "allstudents@cmrit.ac.in"),
    ("viceprincipal@cmrit.ac.in", "allstudents@cmrit.ac.in"),
]

edges_df = pd.DataFrame(edges, columns=["source", "target"])

# Save as CSV
nodes_df.to_csv("nodes.csv", index=False)
edges_df.to_csv("edges.csv", index=False)

print("Dataset saved as nodes.csv and edges.csv")


Dataset saved as nodes.csv and edges.csv


In [16]:
!pip install torch torchvision torchaudio
!pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 212.5/212.5 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 kB 1.9 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.1/1.1 MB 1.6 MB/s eta 0:00:00
     -------------------------------------- 443.7/443.7 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 120.9/120.9 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 45.2/45.2 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 93.4/93.4 kB 2.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd

# Convert email_data to DataFrame
df = pd.DataFrame(email_data)

# Group by sender and calculate features
features = df.groupby("from_email").agg({
    "num_links": "mean",
    "num_attachments": "mean",
    "is_html": "mean",
    "subject": "count"
}).rename(columns={"subject": "email_count"})

features.reset_index(inplace=True)
features.rename(columns={"from_email": "id"}, inplace=True)
features.to_csv("features.csv", index=False)

print("✅ Node features saved to features.csv")


✅ Node features saved to features.csv


In [17]:
import csv

# Example list of emails and their labels (0 for legit, 1 for phishing)
emails = [
    {"id": "iqac@cmrit.ac.in", "label": 0},
    {"id": "viceprincipal@cmrit.ac.in", "label": 0},
    {"id": "no-reply@mail.newsletter.instructables.com", "label": 1},
    {"id": "randomphisher@badmail.com", "label": 1}
]

# Define the output file name
output_file = "labels.csv"

# Write to CSV
with open(output_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["id", "label"])
    writer.writeheader()
    writer.writerows(emails)

print(f"Labels saved to {output_file}")


Labels saved to labels.csv


In [20]:
import torch
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. Load CSVs
nodes_df = pd.read_csv('nodes.csv')
edges_df = pd.read_csv('edges.csv')
features_df = pd.read_csv('features.csv')
labels_df = pd.read_csv('labels.csv')  # Optional

# 2. Encode node IDs as integers
le = LabelEncoder()
nodes_df['node_idx'] = le.fit_transform(nodes_df['id'])
id_map = dict(zip(nodes_df['id'], nodes_df['node_idx']))

# 3. Create edge index (tensor)
edges_df['source'] = edges_df['source'].map(id_map)
edges_df['target'] = edges_df['target'].map(id_map)
edges_df = edges_df.dropna()  # Remove unmapped edges
edge_index = torch.tensor(edges_df[['source', 'target']].values.T, dtype=torch.long)

# 4. Prepare node features
features_df['id'] = features_df['id'].map(id_map)
features_df = features_df.dropna(subset=['id'])  # Drop rows with NaN ID after mapping
features_df = features_df.sort_values('id')
x = torch.tensor(
    StandardScaler().fit_transform(features_df.drop(columns='id').values),
    dtype=torch.float
)

# 5. Prepare labels (ensure they align with features)
labels_df['id'] = labels_df['id'].map(id_map)
labels_df = labels_df.dropna(subset=['id'])  # Drop unmapped
labels_df = labels_df.sort_values('id')

# Align labels to feature nodes only
merged_df = pd.merge(features_df, labels_df, on='id', how='inner')
x = torch.tensor(
    StandardScaler().fit_transform(merged_df.drop(columns=['id', 'label']).values),
    dtype=torch.float
)
y = torch.tensor(merged_df['label'].values, dtype=torch.long)

# 6. Build PyG Data object
data = Data(x=x, edge_index=edge_index, y=y)

# 7. Define a simple GNN
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# 8. Train the GNN
model = GCN(input_dim=x.shape[1], hidden_dim=16, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")


ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required by StandardScaler.